# Initialization

This notebook is built upon its counterpart in the `BNN` folder, and the only difference between the two versions is that this notebook allows a specified amount of dropout to be applied to the training, validation and testing stages of the ensembling process.

Please specify here the amount of dropout you wish to apply to the training, validation and testing stages of the ensembling. A default value of 0.20 is used here.

In [1]:
dropout_amount = 0.20
dropout_int = int(dropout_amount * 100)

In [2]:
%load_ext autoreload
%autoreload
import os, sys
import numpy as np
#Select which GPU to use if necessary
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1
# import mongo
import time
import pickle


env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [3]:
model_name = 'Bayesian CGCNN'

with open('../../preprocessing/sdt/gasdb/feature_dimensions.pkl', 'rb') as file_handle:
    orig_atom_fea_len, nbr_fea_len = pickle.load(file_handle)

with open('../../preprocessing/splits_gasdb.pkl', 'rb') as file_handle:
    splits = pickle.load(file_handle)

docs_train, docs_val, docs_test = splits['docs_train'], splits['docs_val'], splits['docs_test']
sdts_train, sdts_val, sdts_test = splits['sdts_train'], splits['sdts_val'], splits['sdts_test']
targets_train, targets_val, targets_test = splits['targets_train'], splits['targets_val'], splits['targets_test']

This requires a modified version of the CGCNN package, which is available at `https://github.com/ulissigroup/cgcnn/tree/dropout`.  We assume this version of the CGCNN module is installed under the `$HOME/cgcnn` directory. If this is not the case, please modify the first line in the cell below to the directory where CGCNN is installed.

In [4]:
sys.path.append(os.path.expanduser("~/cgcnn"))

from torch.optim import Adam, SGD
from sklearn.model_selection import ShuffleSplit
from skorch.callbacks import Checkpoint, LoadInitState #needs skorch >= 0.4.0
from cgcnn.data import collate_pool, MergeDataset
try:
    from cgcnn.model_pyro import CrystalGraphConvNet # Same model but different names for layers
except ModuleNotFoundError:
    from model_pyro import CrystalGraphConvNet
from skorch import NeuralNetRegressor
import torch
import skorch.callbacks.base

cuda = torch.cuda.is_available()
if cuda:
    device = torch.device("cuda")
else:
    device='cpu'
print('device', device)


# Update dropout value with input
if dropout_amount != 0:
    CrystalGraphConvNet.update_dropout(True, dropout_amount)

device cpu


# Initialize GCNN

In [5]:
from skorch.dataset import CVSplit
from skorch.callbacks.lr_scheduler import WarmRestartLR, LRScheduler
from adamwr.adamw import AdamW
from adamwr.cosine_scheduler import CosineLRWithRestarts
from sklearn.model_selection import train_test_split
from skorch import callbacks
train_test_splitter = ShuffleSplit(test_size=0.25, random_state=42)

#Make a checkpoint to save parameters every time there is a new best for validation lost
cp = Checkpoint(monitor='valid_loss_best',fn_prefix='d%i_valid_best_' % dropout_int)

#Callback to load the checkpoint with the best validation loss at the end of training

class train_end_load_best_valid_loss(skorch.callbacks.base.Callback):
    def on_train_end(self, net, X, y):
        net.load_params('d%i_valid_best_params.pt' % dropout_int)
        
load_best_valid_loss = train_end_load_best_valid_loss()

LR_schedule = callbacks.lr_scheduler.LRScheduler('MultiStepLR', milestones=[100], gamma=0.1)

#############
# To extract intermediate features, set the forward takes only the first return value to calculate loss
class MyNet(NeuralNetRegressor):
    def get_loss(self, y_pred, y_true, **kwargs):
        y_pred = y_pred[0] if isinstance(y_pred, tuple) else y_pred  # discard the 2nd output
        return super().get_loss(y_pred, y_true, **kwargs)
## return features = net.forward(SDT_test)
############

net = MyNet(
    CrystalGraphConvNet,
    module__orig_atom_fea_len = orig_atom_fea_len,
    module__nbr_fea_len = nbr_fea_len,
    batch_size=214,
    module__classification=False,
    lr=0.0056,
    max_epochs= 30, 
    module__atom_fea_len=46,
    module__h_fea_len=83,
    module__n_conv=8, #8
    module__n_h=4,
    optimizer__weight_decay=1e-5,
    optimizer=AdamW,
    iterator_train__pin_memory=True,
    iterator_train__num_workers=0,
    iterator_train__collate_fn = collate_pool,
    iterator_valid__pin_memory=True,
    iterator_valid__num_workers=0,
    iterator_valid__collate_fn = collate_pool,
    device=device,
#     criterion=torch.nn.MSELoss,
    criterion=torch.nn.L1Loss,
    dataset=MergeDataset,
#     train_split = CVSplit(cv=train_test_splitter),
    callbacks=[cp, load_best_valid_loss, LR_schedule]
)

net.initialize()

ModuleNotFoundError: No module named 'adamwr'

# Initialize Pyro model

In [ ]:
import pyro
from pyro.distributions import Normal, Categorical, Uniform
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam, AdamW
pyro.enable_validation(True)

def model(x_data, y_data):
    priors = dict()
    for n, p in net.module_.named_parameters():
        if "fcs" in n:
            i = 2
        elif "conv" in n and "fc_full" in n:
            i = 2
        elif "conv" in n and "bn" in n:
            i = 1

        if "weight" in n:
            priors[n] = pyro.distributions.Normal(
                    loc   = torch.zeros_like(p),
                    scale = torch.ones_like(p)).independent(i)
        elif "bias" in n:
            priors[n] = pyro.distributions.Normal(
                    loc   = torch.zeros_like(p),
                    scale = torch.ones_like(p)).independent(1)
    # lift module parameters to random variables sampled from the priors
    dist = {"fcs_out.weight": priors["fcs_out.weight"], "fcs_out.bias": priors["fcs_out.bias"],
           "fcs2.weight": priors["fcs2.weight"], "fcs2.bias": priors["fcs2.bias"],
           "fcs1.weight": priors["fcs1.weight"], "fcs1.bias": priors["fcs1.bias"],
           "fcs0.weight": priors["fcs0.weight"], "fcs0.bias": priors["fcs0.bias"],
           "fcs_linear.weight": priors["fcs_linear.weight"], "fcs_linear.bias": priors["fcs_linear.bias"]}
    
    
    
    lifted_module = pyro.random_module("module", net.module_, dist)
    # sample a regressor (which also samples w and b)
    lifted_reg_model = lifted_module()
    with pyro.iarange("map", len(x_data)):
        predicted_target = lifted_reg_model(x_data, y_data)
        # condition on the observed data
        pyro.sample("obs",
                    Normal(predicted_target, 0.1*torch.ones_like(torch.tensor(y_data))).independent(2),
                    obs=y_data)

In [ ]:
softplus = torch.nn.Softplus()

def guide(x_data, y_data):

    priors = dict()
    for n, p in net.module_.named_parameters():
        if "fcs" in n:
            i = 2
        elif "conv" in n and "fc_full" in n:
            i = 2
        elif "conv" in n and "bn" in n:
            i = 1

        if "weight" in n:
            loc   = pyro.param("mu_"    + n, torch.randn_like(p))
            scale = torch.abs(pyro.param("sigma_" + n, softplus(torch.randn_like(p))))
            priors[n] = pyro.distributions.Normal(loc = loc, scale = scale).independent(i)
        elif "bias" in n:
            loc       = pyro.param("mu_"    + n, torch.randn_like(p))
            scale     = torch.abs(pyro.param("sigma_" + n, softplus(torch.randn_like(p))))
            priors[n] = pyro.distributions.Normal(loc = loc, scale = scale).independent(1)
                
    dist = {"fcs_out.weight": priors["fcs_out.weight"], "fcs_out.bias": priors["fcs_out.bias"],
           "fcs2.weight": priors["fcs2.weight"], "fcs2.bias": priors["fcs2.bias"],
           "fcs1.weight": priors["fcs1.weight"], "fcs1.bias": priors["fcs1.bias"],
           "fcs0.weight": priors["fcs0.weight"], "fcs0.bias": priors["fcs0.bias"],
           "fcs_linear.weight": priors["fcs_linear.weight"], "fcs_linear.bias": priors["fcs_linear.bias"]}
    lifted_module = pyro.random_module("module", net.module_, dist)
    # sample a regressor (which also samples w and b)
    return lifted_module()

# Training

In [ ]:
batch_size = 48
batch_x_train, batch_y_train = [],[]
batch_train, batch_val, batch_test =[], [], []

for i in range(0, len(sdts_train), batch_size):
    batch_x_train.append(sdts_train[i:i+batch_size])
    batch_y_train.append(targets_train[i:i+batch_size])
    batch_train.append((sdts_train[i:i+batch_size], targets_train[i:i+batch_size]))

batch_x_val, batch_y_val = [],[]
for i in range(0, len(sdts_val), batch_size):
    batch_x_val.append(sdts_val[i:i+batch_size])
    batch_y_val.append(targets_val[i:i+batch_size])
    batch_val.append((sdts_val[i:i+batch_size], targets_val[i:i+batch_size]))
    

batch_x_test, batch_y_test = [],[]
for i in range(0, len(sdts_test), batch_size):
    batch_x_test.append(sdts_test[i:i+batch_size])
    batch_y_test.append(targets_test[i:i+batch_size])
    batch_test.append((sdts_test[i:i+batch_size], targets_test[i:i+batch_size]))

In [ ]:
# import tqdm
# from sklearn.metrics import mean_absolute_error
# import random

# num_iterations = 1000
# loss = 0
# start_loss = 600
# epoch = 0
# lr = 0.01
# # optimizer = Adam({"lr":lr})
# optimizer = AdamW({"lr":lr})

# svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

# for j in range(num_iterations):
#     loss = 0
#     # Shuffle training example each epoch
#     shuffle_idx = np.random.permutation(np.arange(len(sdts_train)))
#     shuffled_sdts_train, shuffled_targets_train = [], []
#     for idx in shuffle_idx:
#         idx = int(idx)
#         shuffled_sdts_train.append(sdts_train[idx])
#         shuffled_targets_train.append(targets_train[idx])
#     shuffled_targets_train = np.array(shuffled_targets_train)
#     batch_train = []
#     for i in range(0, len(shuffled_sdts_train), batch_size):
#         batch_train.append((shuffled_sdts_train[i:i+batch_size], shuffled_targets_train[i:i+batch_size]))

#     for batch_id, data in enumerate(batch_train):
#         loss += svi.step(data[0], torch.tensor(data[1]))
#     normalizer_train = len(sdts_train)
#     total_epoch_loss_train = loss / normalizer_train
    
#     if total_epoch_loss_train < start_loss:
#         print("parameters updated")
#         pyro.get_param_store().save("./param/practice3.save")
#         start_loss = total_epoch_loss_train
        
#     if epoch == 100:
#         lr = lr*0.8
#         print("learning rate decreased")
#         optimizer = Adam({"lr":lr})
#         svi = SVI(model, guide, optimizer, loss=Trace_ELBO())
#         epoch = 0
#     epoch += 1
    
#     if lr <= lr*(0.8)**5:
#         lr = 0.008
#         print("learning rate increased")
#         optimizer = Adam({"lr":lr})
#         svi = SVI(model, guide, optimizer, loss=Trace_ELBO())
    
#     if j % 5 == 0:
#         pred = []
#         true=[]
#         models = guide(None,None)
#         for batch_id, data_test in enumerate(batch_test):
#             pred.append(models(data_test[0], data_test[1]).cpu().detach())
#             true.append(torch.tensor(data_test[1]))

#         MAE = mean_absolute_error(torch.cat(true), torch.cat(pred))
#         print("Epoch ", j, " Loss ", total_epoch_loss_train, "test MAE", MAE)
#     else:
#         print("Epoch ", j, " Loss ", total_epoch_loss_train)


# Load results

In [ ]:
pyro.get_param_store().load("penultimate_BNN.save")
for name in pyro.get_param_store().get_all_param_names():
    print((name, pyro.param(name).data.cpu().numpy()))

# Sampling 
sample networks

In [ ]:
num_samples = 10
sampled_models = [guide(None, None) for _ in range(num_samples)]

# Assess Performance
## Validation

In [ ]:
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm_notebook

predictions_val = []
labels_val = []

for model in tqdm_notebook(sampled_models):
    pred_val = []
    true_val = []
    for batch_id, data_val in enumerate(batch_val):
        pred_val.append(model(data_val[0], data_val[1]).cpu().detach())
        true_val.append(torch.tensor(data_val[1]))
    predictions_val.append(torch.cat(pred_val).numpy())
    labels_val.append(torch.cat(true_val).numpy())
    
predictions_val, labels_val = np.array(predictions_val), np.array(labels_val)

In [ ]:
# from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             r2_score,
                             median_absolute_error)
import seaborn as sns

# Make the predictions
targets_pred = predictions_val.mean(axis=0).flatten()
targets_val = labels_val.mean(axis=0).flatten() 
residuals = (targets_pred - targets_val)#.numpy()
stdevs = predictions_val.std(axis=0)

# Plot
lims = [-4, 4]
grid = sns.jointplot(targets_val.reshape(-1), targets_pred,
                     kind='hex',
                     bins='log',
                     extent=lims+lims)
ax = grid.ax_joint
_ = ax.set_xlim(lims)
_ = ax.set_ylim(lims)
_ = ax.plot(lims, lims, '--')
_ = ax.set_xlabel('DFT $\Delta$E [eV]')
_ = ax.set_ylabel('%s $\Delta$E [eV]' % model_name)

# Calculate the error metrics
mae = mean_absolute_error(targets_val, targets_pred)
rmse = np.sqrt(mean_squared_error(targets_val, targets_pred))
mdae = median_absolute_error(targets_val, targets_pred)
marpd = np.abs(2 * residuals /
               (np.abs(targets_pred) + np.abs(targets_val.reshape(-1)))
               ).mean() * 100
r2 = r2_score(targets_val, targets_pred)
corr = np.corrcoef(targets_val.reshape(-1), targets_pred)[0, 1]

# Report
text = ('    n = %i\n' % len(targets_val) +
        '    MAE = %.2f eV\n' % mae + 
        '    RMSE = %.2f eV\n' % rmse + 
        '    MDAE = %.2f eV\n' % mdae + 
        '    MARPD = %i%%\n' % marpd + 
        '    R$^2$ = %.2f\n' % r2 +
        '    Pearson\'s = %.2f' % corr)
_ = ax.text(x=lims[0], y=lims[1], s=text,
            horizontalalignment='left',
            verticalalignment='top')

In [ ]:
# Calibration

import numpy as np
from scipy import stats
from tqdm import tqdm_notebook

def calculate_density(percentile):
    num_within_quantile = 0
    for stdev, resid in zip(stdevs, residuals):
        norm = stats.norm(loc=0, scale=stdev)
        lower_bound = norm.ppf(0.5-percentile/2)
        upper_bound = norm.ppf(0.5+percentile/2)
        if lower_bound <= resid <= upper_bound:
            num_within_quantile += 1
    density = num_within_quantile / len(residuals)
    return density

predicted_pi = np.linspace(0, 1, 20)
observed_pi = [calculate_density(quantile)
               for quantile in tqdm_notebook(predicted_pi, desc='Calibration')]

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from shapely.geometry import Polygon


# Plot settings
figsize = (4, 4)

# Plot the calibration curve
fig_cal = plt.figure(figsize=figsize)
ax_ideal = sns.lineplot([0, 1], [0, 1], label='ideal')
_ = ax_ideal.lines[0].set_linestyle('--')
ax_gp = sns.lineplot(predicted_pi, observed_pi, label=model_name)
ax_fill = plt.fill_between(predicted_pi, predicted_pi, observed_pi,
                           alpha=0.2, label='miscalibration area')
_ = ax_ideal.set_xlabel('Expected prediction interval')
_ = ax_ideal.set_ylabel('Observed prediction interval')
_ = ax_ideal.set_xlim([0, 1])
_ = ax_ideal.set_ylim([0, 1])

# Calculate the miscalibration area.
polygon_points = []
for point in zip(predicted_pi, observed_pi):
    polygon_points.append(point)
for point in zip(reversed(predicted_pi), reversed(predicted_pi)):
    polygon_points.append(point)
polygon_points.append((predicted_pi[0], observed_pi[0]))
polygon = Polygon(polygon_points)
miscalibration_area = polygon.area

# Annotate the plot with the miscalibration area
plt.text(x=0.95, y=0.05,
         s='Miscalibration area = %.2f' % miscalibration_area,
         verticalalignment='bottom',
         horizontalalignment='right')


# Plot sharpness curve
xlim = [0, 1.]
fig_sharp = plt.figure(figsize=figsize)
ax_sharp = sns.distplot(stdevs, kde=False, norm_hist=True)
ax_sharp.set_xlim(xlim)
ax_sharp.set_xlabel('Predicted standard deviation (eV)')
ax_sharp.set_ylabel('Normalized frequency')
ax_sharp.set_yticklabels([])
ax_sharp.set_yticks([])

# Calculate and report sharpness
sharpness = np.sqrt(np.mean(stdevs**2))
_ = ax_sharp.axvline(x=sharpness, label='sharpness')
if sharpness < (xlim[0] + xlim[1]) / 2:
    text = '\n  Sharpness = %.2f eV' % sharpness
    h_align = 'left'
else:
    text = '\nSharpness = %.2f eV  ' % sharpness
    h_align = 'right'
_ = ax_sharp.text(x=sharpness, y=ax_sharp.get_ylim()[1],
                  s=text,
                  verticalalignment='top',
                  horizontalalignment=h_align)

# Test

In [ ]:
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm_notebook

predictions_test = []
labels_test = []
for model in tqdm_notebook(sampled_models):
    pred_test = []
    true_test = []
    for batch_id, data_test in enumerate(batch_test):
        pred_test.append(model(data_test[0], data_test[1]).cpu().detach())
        true_test.append(torch.tensor(data_test[1]))
    predictions_test.append(torch.cat(pred_test).numpy())
    labels_test.append(torch.cat(true_test).numpy())

predictions_test, labels_test = np.array(predictions_test), np.array(labels_test)

In [ ]:
# from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             r2_score,
                             median_absolute_error)
import seaborn as sns

# Make the predictions
targets_pred = predictions_test.mean(axis=0).flatten()
targets_val = labels_test.mean(axis=0).flatten() 
residuals = (targets_pred - targets_val)#.numpy()
stdevs = predictions_test.std(axis=0)

# Plot
lims = [-4, 4]
grid = sns.jointplot(targets_val.reshape(-1), targets_pred,
                     kind='hex',
                     bins='log',
                     extent=lims+lims)
ax = grid.ax_joint
_ = ax.set_xlim(lims)
_ = ax.set_ylim(lims)
_ = ax.plot(lims, lims, '--')
_ = ax.set_xlabel('DFT $\Delta$E [eV]')
_ = ax.set_ylabel('%s $\Delta$E [eV]' % model_name)

# Calculate the error metrics
mae = mean_absolute_error(targets_val, targets_pred)
rmse = np.sqrt(mean_squared_error(targets_val, targets_pred))
mdae = median_absolute_error(targets_val, targets_pred)
marpd = np.abs(2 * residuals /
               (np.abs(targets_pred) + np.abs(targets_val.reshape(-1)))
               ).mean() * 100
r2 = r2_score(targets_val, targets_pred)
corr = np.corrcoef(targets_val.reshape(-1), targets_pred)[0, 1]

# Report
text = ('    n = %i\n' % len(targets_val) +
        '    MAE = %.2f eV\n' % mae + 
        '    RMSE = %.2f eV\n' % rmse + 
        '    MDAE = %.2f eV\n' % mdae + 
        '    MARPD = %i%%\n' % marpd + 
        '    R$^2$ = %.2f\n' % r2 +
        '    Pearson\'s = %.2f' % corr)
_ = ax.text(x=lims[0], y=lims[1], s=text,
            horizontalalignment='left',
            verticalalignment='top')

In [ ]:
# Calibration

import numpy as np
from scipy import stats
from tqdm import tqdm_notebook

def calculate_density(percentile):
    num_within_quantile = 0
    for stdev, resid in zip(stdevs, residuals):
        norm = stats.norm(loc=0, scale=stdev)
        lower_bound = norm.ppf(0.5-percentile/2)
        upper_bound = norm.ppf(0.5+percentile/2)
        if lower_bound <= resid <= upper_bound:
            num_within_quantile += 1
    density = num_within_quantile / len(residuals)
    return density

predicted_pi = np.linspace(0, 1, 20)
observed_pi = [calculate_density(quantile)
               for quantile in tqdm_notebook(predicted_pi, desc='Calibration')]

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from shapely.geometry import Polygon


# Plot settings
figsize = (4, 4)

# Plot the calibration curve
fig_cal = plt.figure(figsize=figsize)
ax_ideal = sns.lineplot([0, 1], [0, 1], label='ideal')
_ = ax_ideal.lines[0].set_linestyle('--')
ax_gp = sns.lineplot(predicted_pi, observed_pi, label=model_name)
ax_fill = plt.fill_between(predicted_pi, predicted_pi, observed_pi,
                           alpha=0.2, label='miscalibration area')
_ = ax_ideal.set_xlabel('Expected prediction interval')
_ = ax_ideal.set_ylabel('Observed prediction interval')
_ = ax_ideal.set_xlim([0, 1])
_ = ax_ideal.set_ylim([0, 1])

# Calculate the miscalibration area.
polygon_points = []
for point in zip(predicted_pi, observed_pi):
    polygon_points.append(point)
for point in zip(reversed(predicted_pi), reversed(predicted_pi)):
    polygon_points.append(point)
polygon_points.append((predicted_pi[0], observed_pi[0]))
polygon = Polygon(polygon_points)
miscalibration_area = polygon.area

# Annotate the plot with the miscalibration area
plt.text(x=0.95, y=0.05,
         s='Miscalibration area = %.2f' % miscalibration_area,
         verticalalignment='bottom',
         horizontalalignment='right')


# Plot sharpness curve
xlim = [0, 1.]
fig_sharp = plt.figure(figsize=figsize)
ax_sharp = sns.distplot(stdevs, kde=False, norm_hist=True)
ax_sharp.set_xlim(xlim)
ax_sharp.set_xlabel('Predicted standard deviation (eV)')
ax_sharp.set_ylabel('Normalized frequency')
ax_sharp.set_yticklabels([])
ax_sharp.set_yticks([])

# Calculate and report sharpness
sharpness = np.sqrt(np.mean(stdevs**2))
_ = ax_sharp.axvline(x=sharpness, label='sharpness')
if sharpness < (xlim[0] + xlim[1]) / 2:
    text = '\n  Sharpness = %.2f eV' % sharpness
    h_align = 'left'
else:
    text = '\nSharpness = %.2f eV  ' % sharpness
    h_align = 'right'
_ = ax_sharp.text(x=sharpness, y=ax_sharp.get_ylim()[1],
                  s=text,
                  verticalalignment='top',
                  horizontalalignment=h_align)

# Error-bar figure
We concede that calibration curves and sharpness distributions are new concepts in the field of catalysis, and that a simple parity plot with error bars is more intuitive. As such, we create a few examples of error bar parities to help readers connect those incumbent ideas with the newer ideas of calibration and sharpness.|

In [ ]:
%matplotlib inline
import random
import numpy as np
import matplotlib.pyplot as plt


# Pull a random sample of the data, because plotting thousands of these at once would look absurd
all_predictions = list(zip(targets_pred, targets_test.reshape(-1), stdevs))
samples = random.sample(all_predictions, k=20)

# Parse the samples
_preds, _targets, _stdevs = zip(*samples)
_preds = np.array(_preds)
_targets = np.array(_targets)
_stdevs = np.array(_stdevs)

# Plot
fig = plt.figure(figsize=(4, 4))
_ = plt.errorbar(_targets, _preds, yerr=2*_stdevs, fmt='o')
ax = plt.gca()

# Make a parity line
lims = [-2, 2]
_ = ax.plot(lims, lims, '--')

# Format
_ = ax.set_xlim(lims)
_ = ax.set_ylim(lims)
_ = ax.set_xticks(list(range(-2, 3)))
_ = ax.set_yticks(list(range(-2, 3)))
_ = ax.set_xlabel('DFT $\Delta$E [eV]')
_ = ax.set_ylabel('%s $\Delta$E [eV]' % model_name)

# Save
_ = plt.savefig('error_bar_parity.pdf', dpi=300, bbox_inches='tight', transparent=True)

## Saving the data
We need to save the prediction data with the `pickle` module, so we can compare the effects of different prediction methods and different dropout levels in one plot, using another notebook.

In [ ]:
# Save as pickle to be plotted with in the same graph as others
with open('bnn_old_d%i.pkl' % dropout_int, 'wb') as saveplot:
    pickle.dump((predictions, targets_val), saveplot)